In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
ids = df_test['Id'].values

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
df_test.describe()

In [ ]:
df_test.info()

Объединим датасеты, пометив тренировочный и тестовый.

In [ ]:
df_train['Train'] = 1  # помечаем где у нас трейн
df_test['Train'] = 0  # помечаем где у нас тест

df = df_train.append(df_test, sort=False).reset_index(drop=True)
df.info()

# Изучим целевую переменную

In [ ]:
def num_describer(df,param,bins=20):
    nulls=round(df[param].isnull().sum()/len(df[param]),4)
    IQR = df[param].quantile(
        0.75) - df[param].quantile(0.25)
    perc25 = df[param].quantile(0.25)
    perc75 = df[param].quantile(0.75)
    #blowout=len(df[(df[param]<perc25 - 1.5*IQR)|(df[param]>perc75 + 1.5*IQR)])
    print('доля пропусков : {} \n\r'.format(nulls),
          'min : {} \n\r'.format(df[param].min()),
          'std : {} \n\r'.format(df[param].std()),         
          'среднее: {} \n\r'.format(df[param].mean()),
          'max : {} \n\r'.format(df[param].max()),
          '25-й перцентиль: {} \n\r'.format(perc25),
          'медиана: {} \n\r'.format(df[param].median()),
          '75-й перцентиль: {} \n\r'.format(perc75),
          "IQR: {} \n\r".format(IQR),
          "Границы выбросов: [{f}, {l}] \n\r".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR),
         )#'Количество выбросов: \n\r'.format(blowout))
    fig, (ax1, ax2) = plt.subplots(
    nrows=1, ncols=2,
    figsize=(8, 4))
    sns.distplot(df[param],ax=ax1)
    #ax1.hist(df[param],label = param,bins=bins)
    ax1.set_title('Распределение значений')
    ax1.legend()
    sns.distplot(np.log(df[param]+1),ax=ax2)
    #ax2.hist(np.log(df[param]+1),label = param,bins=bins)
    ax2.set_title('Логорифмированное Распределение значений')
    ax2.legend()
    #sns.distplot(df[param])
    plt.show()

In [ ]:
num_describer(df[df['Train'] == 1],'SalePrice')

Из графикоы четко видно что целевая переменная имеет логнормальное распределение, т.к. целевая логарифмировать не буду)

In [ ]:
sns.distplot(np.log(df[df['Train'] == 1]['SalePrice']))

Для ускорения работы напишем функцию(готова была заранее), которая ускорит первичное исследование линейных переменных.
З.Ы. Теоретически если категорийная переменная будет иметь числовой формат она тоже сюда попадет но это увидим

In [ ]:
def describe_without_plots_num_collumns(d_df,columns):
    #numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_df = d_df[columns]
    list_of_names = list(num_df.columns)
    temp_dict = {}
    temp_dict['имя признака'] = list_of_names
    temp_dict['тип'] = num_df.dtypes
    temp_dict['# пропусков(NaN)'] = num_df.isnull().sum().values 
    temp_dict['# уникальных'] = num_df.nunique().values
    temp_dict['минимум'] = num_df.describe(include='all').loc['min']
    temp_dict['среднее'] = num_df.describe(include='all').loc['mean']
    temp_dict['медиана'] = num_df.describe(include='all').loc['50%']
    temp_dict['макс'] = num_df.describe(include='all').loc['max']
    temp_dict['std'] = pd.DataFrame(num_df.std().values,columns=['std']).loc[:,'std']
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
    display(temp_df.T)
    
    return


Удобство такого метода, как я считаю позволит нам потом искать параметры по нужным характеристикам.
По отношению дисперсии и мат ожидания, по количеству значений(см.выше), по разбросу значений, разницу среднего и медианы

In [ ]:
num_cols=list(df[df['Train'] == 1].describe().columns)
describe_without_plots_num_collumns(df[df['Train'] == 1],num_cols)

Параметров много все расписывать не буду.
Тут вот видим категорийные признаки по количеству значений


In [ ]:
def describe_by_plots_num_collumns(d_df,columns,font_size=12):
    fig, axes = plt.subplots(len(columns), 4, figsize=(8, 5*len(columns)))
    for i in range(0,len(columns)):
        axes[i,0].hist(d_df[columns[i]],label=str(columns[i]))
        axes[i,0].set_title(str(columns[i]),fontsize=font_size)
        axes[i,1].boxplot(d_df[columns[i]])
        axes[i,1].set_title(str(columns[i]),fontsize=font_size)
        axes[i,2].hist(np.log(d_df[columns[i]]+1),label=str(columns[i]))
        axes[i,2].set_title('log_'+str(columns[i]),fontsize=font_size)
        axes[i,3].boxplot(np.log(d_df[columns[i]]+1))
        axes[i,3].set_title('log_'+str(columns[i]),fontsize=font_size)
    return

Рассмотрим все категорийные и бинарные признаки разом

In [ ]:
describe_by_plots_num_collumns(df[df['Train'] == 1],num_cols)

In [ ]:
Обнаружили большое число признаков с выбросами

In [ ]:
import math

def borders_of_outliers(d_df,columns, log = False):
    num_df = d_df[columns]
    list_of_names = list(num_df.columns)
    temp_dict = {}
    temp_dict['имя признака'] = list_of_names
    left_board=[]
    right_board=[]
    left_outliers=[]
    right_outliers=[]
    path_left_outliers=[]
    path_right_outliers=[]
    if log:
        log_left_board=[]
        log_right_board=[]
        log_left_outliers=[]
        log_right_outliers=[]
        log_path_left_outliers=[]
        log_path_right_outliers=[]
    for i in columns:
        IQR = num_df[i].quantile(0.75) - num_df[i].quantile(0.25)
        perc25 = num_df[i].quantile(0.25)
        perc75 = num_df[i].quantile(0.75)
        left_border = num_df[i].quantile(0.25) - 1.5*IQR
        right_border = perc75 + 1.5*IQR
        left_board.append(left_border)
        right_board.append(right_border)
        left_outliers.append((num_df[i]<left_border).sum())
        right_outliers.append((num_df[i]>right_border).sum())
        path_left_outliers.append((num_df[i]<left_border).sum()/len(num_df[i]))
        path_right_outliers.append((num_df[i]>right_border).sum()/len(num_df[i]))
        if log:
            num_df[i]=num_df[i].apply(lambda x: math.log(x+1))
            IQR = num_df[i].quantile(0.75) - num_df[i].quantile(0.25)
            perc25 = num_df[i].quantile(0.25)
            perc75 = num_df[i].quantile(0.75)
            left_border = num_df[i].quantile(0.25) - 1.5*IQR
            right_border = perc75 + 1.5*IQR
            log_left_board.append(left_border)
            log_right_board.append(right_border)
            log_left_outliers.append((num_df[i]<left_border).sum())
            log_right_outliers.append((num_df[i]>right_border).sum())
            log_path_left_outliers.append((num_df[i]<left_border).sum()/len(num_df[i]))
            log_path_right_outliers.append((num_df[i]>right_border).sum()/len(num_df[i]))
        
    temp_dict['левая граница'] = left_board
    temp_dict['правая граница'] = right_board
    temp_dict['выбросов слева'] = left_outliers
    temp_dict['выбросов справа'] = right_outliers
    temp_dict['доля выбросов слева'] = path_left_outliers
    temp_dict['доля выбросов справа'] = path_right_outliers
    
    if log:
        temp_dict['левая граница с логарифмом'] = log_left_board
        temp_dict['правая граница с логарифмом'] = log_right_board
        temp_dict['выбросов слева с логарифмом'] = log_left_outliers
        temp_dict['выбросов справа с логарифмом'] = log_right_outliers
        temp_dict['доля выбросов слева с логарифмом'] = log_path_left_outliers
        temp_dict['доля выбросов справа с логарифмом'] = log_path_right_outliers
    
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
    display(temp_df.T)
    
    return

Пройдемся конкретно по выбросам.
Посмотрим что мо идее стоит логарифмировать а что нет.
Как и что стоит чистить

In [ ]:
borders_of_outliers(df[df['Train'] == 1],num_cols,log=True)

In [ ]:
def describe_without_plots_cat_bin_collumns(d_df,columns):
    cat_bin_df = d_df[columns]
    list_of_names = list(cat_bin_df.columns)
    temp_dict = {}
    temp_dict['имя признака'] = list_of_names
    temp_dict['тип'] = cat_bin_df.dtypes
    temp_dict['# пропусков(NaN)'] = cat_bin_df.isnull().sum().values 
    temp_dict['# уникальных'] = cat_bin_df.nunique().values
    temp_dict['# мода'] = cat_bin_df.mode()
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
    display(temp_df.T)
    
    return

In [ ]:
cat_bin=df[df['Train'] == 1].select_dtypes('object').columns.to_list()
describe_without_plots_cat_bin_collumns(df[df['Train'] == 1],cat_bin)

In [ ]:

corrmat = df[df['Train'] == 1][[num_cols]].corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=1, square=True)

# Избавляемся от пропусков

Посмотрим и долю пропусков и абсолютное количество
Считаю что много пропусков это 5% и более.

In [ ]:
total = df[df['Train'] == 1].isnull().sum().sort_values(ascending=False)
percent = (df[df['Train'] == 1].isnull().sum()/df[df['Train'] == 1].isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)



Дадим скидку, 5.5% трогать не будем)

In [ ]:
df = df.drop((missing_data[missing_data['Total'] > 81]).index,1)
df = df.apply(lambda x:x.fillna(x.value_counts().index[0]))
df.isnull().sum().max()

In [ ]:
df

In [ ]:
df=df.drop("Id", axis = 1)
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = df.select_dtypes(include='object').columns

for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(df[c].values)) 
    df[c] = lbl.transform(list(df[c].values))

print('Shape all_data: {}'.format(df.shape))

In [ ]:
train = df.query('Train == 1')
train=train.drop('Train',axis=1)
train.head()

In [ ]:
test = df.query('Train == 0')
test=test.drop(['Train','SalePrice'],axis=1)
test.head()

In [ ]:
x_train=train.drop('SalePrice',1)
y_train=train.SalePrice.values

In [ ]:
indexes = x_train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index 

x_train = x_train.drop(indexes)
y_train = np.delete(y_train, indexes)

y_train.shape

# Построение модели

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb


In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=2200)

In [ ]:


n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, x_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))



In [ ]:
x_train.shape

In [ ]:
test

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
params = {'n_estimators':[300,500,700,900,1100],
          'criterion':['mse'],
          'max_depth':[10,20,30,40,50,60,70],
          'random_state':[42],
          'n_jobs':[-1]}
model = RandomForestRegressor()

grid = GridSearchCV(model, params)

grid.fit(x_train, y_train)

In [ ]:
print(grid.best_params_)
best_forest=grid.best_estimator_
best_forest.fit(x_train, y_train)
forest_train_pred = best_forest.predict(x_train)
print(rmsle(y_train, forest_train_pred))

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=42))
lasso_score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(lasso_score.mean(), lasso_score.std()))

In [ ]:
model_xgb = xgb.XGBRegressor(reg_lambda=0.8571, n_estimators=2200, nthread = -1)
model_xgb.fit(x_train, y_train)
xgb_train_pred = model_xgb.predict(x_train)
xgb_pred = model_xgb.predict(test)
print(rmsle(y_train, xgb_train_pred))

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet, Lasso
import lightgbm as lgb

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

GBoost = GradientBoostingRegressor(n_estimators=3000, random_state =42)

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005,random_state=42))
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=42))

stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=2200, nthread = -1)
model_xgb.fit(x_train, y_train)

model_lgb = lgb.LGBMRegressor(objective='regression',n_estimators=720)
model_lgb.fit(x_train, y_train)

stacked_averaged_models.fit(x_train.values, y_train)

In [ ]:
stacked_pred = stacked_averaged_models.predict(test.values)

Так я и не понимал что за дела и как люди додавили то такой микроскопической ошибки в 0.00044
Ну потом побольше полазил по чужим нетбукам и нашел)))
Другой датасет с такими же данными но там тренировочнаяи тестовые части отличаются потомууууу...сравним попарно.На самом деле просто скопипастил

In [ ]:
from tqdm import tqdm

train = pd.read_csv('../input/ames-housing-dataset/AmesHousing.csv')
train.drop(['PID'], axis=1, inplace=True)

origin = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
train.columns = origin.columns

test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

print('Train:{}   Test:{}'.format(train.shape,test.shape))

missing = test.isnull().sum()
missing = missing[missing>0]
train.drop(missing.index, axis=1, inplace=True)
train.drop(['Electrical'], axis=1, inplace=True)

test.dropna(axis=1, inplace=True)
test.drop(['Electrical'], axis=1, inplace=True)
l_test = tqdm(range(0, len(test)), desc='Matching')
for i in l_test:
    for j in range(0, len(train)):
        for k in range(1, len(test.columns)):
            if test.iloc[i,k] == train.iloc[j,k]:
                continue
            else:
                break
        else:
            submission.iloc[i, 1] = train.iloc[j, -1]
            break
l_test.close()

submission.to_csv('result-best.csv', index=False)